In [24]:
from pyspark.sql import SparkSession

In [25]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

In [26]:
user = "NBABrasil"

In [27]:
df_users = spark.read.json("../../datalake/Silver/twitter_nbabrasil/user")

In [28]:
user_id = df_users.filter(f'username = "{user}"').select("id").first()[0]
user_id

'589433710'

In [29]:
df_tweet = spark.read.json("../../datalake/Silver/twitter_nbabrasil/tweet")

In [30]:
nba_brasil = df_tweet\
        .where(f"author_id = '{user_id}'")\
        .select("author_id", "conversation_id")

In [31]:
from pyspark.sql.functions import when, col, sum, to_date, date_format, countDistinct

In [32]:
tweet_conversas = df_tweet.alias("tweet")\
        .join(
            nba_brasil.alias("nba_brasil"),
            [
                nba_brasil.author_id != df_tweet.author_id,
                nba_brasil.conversation_id == df_tweet.conversation_id
            ],
            'left')\
        .withColumn(
            "nba_brasil_conversation",
            when(col("nba_brasil.conversation_id").isNotNull(), 1).otherwise(0)
        ).withColumn(
            "reply_nba_brasil",
            when(col("tweet.in_reply_to_user_id") == user_id, 1).otherwise(0)
        ).groupBy(to_date("created_at").alias("created_date"))\
        .agg(
            countDistinct("id").alias("n_tweets"),
            countDistinct("tweet.conversation_id").alias("n_conversation"),
            sum("nba_brasil_conversation").alias("nba_brasil_conversation"),
            sum("reply_nba_brasil").alias("reply_alura")
        ).withColumn("weekday", date_format("created_date", "E"))

In [34]:
tweet_conversas.orderBy("created_date").show()

+------------+--------+--------------+-----------------------+-----------+-------+
|created_date|n_tweets|n_conversation|nba_brasil_conversation|reply_alura|weekday|
+------------+--------+--------------+-----------------------+-----------+-------+
|  2022-08-10|     137|           104|                     32|         36|    Wed|
|  2022-08-11|     226|           157|                     69|         56|    Thu|
|  2022-08-12|     116|           110|                      7|          9|    Fri|
|  2022-08-13|      50|            42|                      8|         10|    Sat|
|  2022-08-14|     170|           148|                      9|         15|    Sun|
|  2022-08-15|     390|           275|                    384|        206|    Mon|
|  2022-08-16|     265|           222|                    140|         93|    Tue|
|  2022-08-17|     266|           216|                    478|        378|    Wed|
|  2022-08-18|     184|           160|                     75|         62|    Thu|
|  2